### Import Relevant Libraries

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
from numpy.random import seed
seed(1)
#from tensorflow import random
#random.set_seed(1)
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets
from tensorflow.keras.utils import to_categorical

### Load Data

In [3]:
covid_deaths_county = pd.read_csv("Step0_dataCollection/April_18_data.csv")
df2 = pd.read_csv("Step0_dataCollection/Age_Final.csv")

### Clean Data

In [4]:
df1 = covid_deaths_county.copy()
df1 = df1.rename(columns={'county':'County'})

In [5]:
df1.head()

,County,State,FIPS,Apr18_Cases,Apri18_Deaths
0,Autauga,Alabama,1001,25,2
1,Baldwin,Alabama,1003,109,2
2,Barbour,Alabama,1005,18,0
3,Bibb,Alabama,1007,26,0
4,Blount,Alabama,1009,20,0


In [6]:
df2 = df2.rename(columns={'county':'County', 'fips':"FIPS"})
df2.head()

,state,County,% Age 0 to 29 years,% Age 30 to 39,% Age 40 to 49 years,% Age 50 to 59 years,% Age 60 to 69 years,% Age 70 to 79,% Age 80 to 85+,FIPS
0,Alabama,Autauga,38.312261,12.749771,13.636445,13.917016,10.478229,7.284042,3.622237,1001
1,Alabama,Baldwin,34.514407,11.564429,12.539102,13.753658,13.728890,9.602701,4.296814,1003
2,Alabama,Barbour,36.011414,12.865239,12.354809,13.174712,12.362847,9.018930,4.212049,1005
3,Alabama,Bibb,36.075893,13.897321,13.562500,13.906250,11.361607,7.691964,3.504464,1007
4,Alabama,Blount,37.010719,11.901798,12.921853,13.561549,12.041840,8.525242,4.036999,1009


In [7]:
raw_data = pd.merge(df1, df2, how='left', on='FIPS')

In [8]:
raw_data = raw_data.dropna()
raw_data

,County,State,FIPS,Apr18_Cases,Apri18_Deaths,state,County,% Age 0 to 29 years,% Age 30 to 39,% Age 40 to 49 years,% Age 50 to 59 years,% Age 60 to 69 years,% Age 70 to 79,% Age 80 to 85+
0,Autauga,Alabama,1001,25,2,Alabama,Autauga,38.312261,12.749771,13.636445,13.917016,10.478229,7.284042,3.622237
1,Baldwin,Alabama,1003,109,2,Alabama,Baldwin,34.514407,11.564429,12.539102,13.753658,13.728890,9.602701,4.296814
2,Barbour,Alabama,1005,18,0,Alabama,Barbour,36.011414,12.865239,12.354809,13.174712,12.362847,9.018930,4.212049
3,Bibb,Alabama,1007,26,0,Alabama,Bibb,36.075893,13.897321,13.562500,13.906250,11.361607,7.691964,3.504464
4,Blount,Alabama,1009,20,0,Alabama,Blount,37.010719,11.901798,12.921853,13.561549,12.041840,8.525242,4.036999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,Sublette,Wyoming,56035,1,0,Wyoming,Sublette,33.822480,12.799348,12.942016,13.573831,13.981453,9.222460,3.658412
2710,Sweetwater,Wyoming,56037,10,0,Wyoming,Sweetwater,41.320759,15.277229,12.389956,12.220390,11.481731,5.105572,2.204362
2711,Teton,Wyoming,56039,62,0,Wyoming,Teton,32.650232,17.733200,14.830380,12.685759,12.456133,6.797799,2.846497
2712,Uinta,Wyoming,56041,6,0,Wyoming,Uinta,41.770531,13.311001,11.773979,11.990738,12.581901,6.074191,2.497660


In [9]:
list(raw_data.columns)

['County ',
 'State',
 'FIPS',
 'Apr18_Cases',
 'Apri18_Deaths',
 'state',
 'County',
 '% Age 0 to 29 years',
 '% Age 30 to 39',
 '% Age 40 to 49 years',
 '% Age 50 to 59 years',
 '% Age 60 to 69 years',
 '% Age 70 to 79',
 '% Age 80 to 85+']

In [40]:
raw_data = raw_data.drop(columns=['fips','SUMLEV','STATE','Unnamed: 2','STNAME','YEAR'])

In [41]:
raw_data

,date,County,state,cases,deaths,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE
0,2020-01-21,Snohomish,Washington,1,0,0.0,814901.0,408857.0,406044.0
1,2020-01-21,Snohomish,Washington,1,0,1.0,52083.0,26942.0,25141.0
2,2020-01-21,Snohomish,Washington,1,0,2.0,50976.0,26232.0,24744.0
3,2020-01-21,Snohomish,Washington,1,0,3.0,51514.0,26485.0,25029.0
4,2020-01-21,Snohomish,Washington,1,0,4.0,46683.0,24161.0,22522.0
...,...,...,...,...,...,...,...,...,...
6489816,2020-04-18,Washakie,Wyoming,4,0,14.0,517.0,268.0,249.0
6489817,2020-04-18,Washakie,Wyoming,4,0,15.0,420.0,204.0,216.0
6489818,2020-04-18,Washakie,Wyoming,4,0,16.0,321.0,175.0,146.0
6489819,2020-04-18,Washakie,Wyoming,4,0,17.0,210.0,84.0,126.0


### Choose dependent and independent variables

In [72]:
y = raw_data[['cases']]
X = raw_data[['AGEGRP','TOT_POP']]

### Split Data Set

In [73]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size= 0.3)

### Standardize Data Set

In [74]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)
# Transform the training and testing data using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Create Model

In [75]:
# first, create a normal neural network with 2 inputs, 6 hidden nodes, and 2 outputs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=6, activation='sigmoid', input_dim=2))
model.add(Dense(units=4, activation='sigmoid'))

In [76]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 6)                 18        
_________________________________________________________________
dense_11 (Dense)             (None, 4)                 28        
Total params: 46
Trainable params: 46
Non-trainable params: 0
_________________________________________________________________


In [82]:
# Compile the model
model.compile(optimizer='adam',
              loss= 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=4
)

Train on 4542216 samples
Epoch 1/300


InvalidArgumentError:  Received a label value of 209 which is outside the valid range of [0, 4).  Label values: 209 1 1 1 3 37 1 33 7 1 13 82 4 16 4 14 26 1 95 24 12 7 5 68 34 15 92 7 1 25 10 6
	 [[node loss/dense_11_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-83-a4f9d8bc9726>:7) ]] [Op:__inference_distributed_function_1070]

Function call stack:
distributed_function
